In [ ]:
from transformers import pipeline
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd
pd.set_option('display.max_rows', None)

## Setup

In [ ]:
context = """
Elizabeth 2. var dronning af Storbritannien fra 1952 til 2022. Hun var ældste datter af George 6. og dronning Elizabeth.

Elizabeth blev som kronprinsesse i 1947 gift med løjtnant Philip Mountbatten, tidligere prins Philip af Grækenland og Danmark og efter giftermålet hertug af Edinburgh. Philip døde 9. april 2021. Parret fik fire børn, Charles, Anne, Andrew og Edward.

Elizabeth var kun 25 år gammel, da hun overtog tronen efter George 6.s død den 6. februar 1952. Den formelle kroningsceremoni fandt dog først sted den 2. juni 1953.

Dronningens reelle magt i det britiske politiske system var beskeden, men symbolsk har hun haft stor betydning, også i de dele af Commonwealth, som hun også var statsoverhoved for.

I dagene omkring den 2. juni 2022 fejrede Storbritannien Elizabeths 70 år på tronen. Hun er den længst siddende britiske monark nogensinde og den næst længst siddende i verden. Elizabeth måtte dog fravælge deltagelse i en del arrangementer i forbindelse med platinjubilæet pga. svigtende helbred.

Det britiske monarkis store popularitet hænger nøje sammen med den måde, hvorpå Elizabeth med stor pligtopfyldelse udførte sine krævende ceremonielle og repræsentative pligter. Hun stod således som bolværk mod angreb på monarkiet i nogle turbulente årtier med skandaler og skilsmisser i den britiske royale familie, og det har i højere grad været personen Elizabeth end monarkiet som sådan, der har været årsagen til dets popularitet i den britiske befolkning. I en tid, hvor de fleste andre bærende institutioner i det britiske samfund har været eller er under pres og forandring, har Elizabeth stået for en meget langvarig stabilitet og en forbindelse til en svunden tid, som bl.a. Brexitafstemningen i 2016 viste, at mange briter stadig har veneration for.

Hendes holdning i forbindelse med prinsesse Dianas død og bisættelse i efteråret 1997 vakte dog stor uro i den britiske befolkning og belystes i 2006 i Stephen Frears' spillefilm The Queen, hvor hun blev portrætteret af Helen Mirren. Også Netflixserien The Crown portrætterer i fiktionsform i foreløbig fire sæsoner monarkiets og det britiske samfunds udvikling fra 1950erne og frem til foreløbig 1990erne.

Dronningen ejede en af de betydeligste private kunstsamlinger i verden og rådede over store jordbesiddelser og en meget stor formue, hvoraf en del, som afdækket af de såkaldte Panama Papers, er placeret i skattely.

Ved hendes død den 8. september 2022 blev hendes ældste søn, Charles, konge.
"""

In [ ]:
questions = [
    "Hvornår døde Dronning Elizabeth?",
    "Hvad hedder Dronning Elizabeth's forældre?",
    "Hvad hedder Dronning Elizabeth's far?",
    "Hvem er nu regent efter Elizabeth?",
    "Hvem er nu konge efter Elizabeth?",
    "Hvor gammel var Elizabeth, da hun blev dronning?",
    "Hvem spillede Elizabeth i 'The Queen'?",
    "Hvad hedder Dronning Elizabeth's børn?",
    "I hvilken periode foregår serien 'The Crown'?",
    "Hvor blev Dronning Elizabeth begravet?",
    "I hvilken by blev Dronning Elizabeth født?",
    "Hvad var Dronning Elizabeth's holdning til Brexit?",
]

## Inference with models

In [ ]:
def print_model_answers(model_id: str) -> None:
    qa = pipeline(task='question-answering', model=model_id, use_auth_token=True)
    for question in questions:
        output = qa(context=context, question=question, max_answer_len=15, handle_impossible_answer=True, top_k=3)
        answers = [dct["answer"] for dct in output]
        print(f"Question: {question!r}")
        print("Model's answers:")
        for idx, answer in enumerate(answers):
            print(f"\t{1 + idx}. {answer!r}")
        print()

In [ ]:
model_ids = [
    "saattrupdan/xlm-roberta-base-qa-scandi",
    "saattrupdan/xlm-roberta-large-qa-scandi",
    "saattrupdan/xlm-roberta-large-qa-sv-scandi",
    "jacobshein/danish-bert-botxo-qa-squad",
    "saattrupdan/xlmr-base-texas-squad-da",
]

In [ ]:
for model_id in model_ids:
    print(f"{' ' + model_id + ' ':=^50}")
    print()
    print_model_answers(model_id=model_id)
    print()

## Error analysis

In [ ]:
texas_qa = pipeline(task='question-answering', model="saattrupdan/xlmr-base-texas-squad-da", use_auth_token=True)
botxo_qa = pipeline(task='question-answering', model="jacobshein/danish-bert-botxo-qa-squad", use_auth_token=True)
scandi_qa = pipeline(task='question-answering', model="saattrupdan/xlm-roberta-base-qa-scandi", use_auth_token=True)

In [ ]:
val_ds = load_dataset("alexandrainst/scandiqa", "da", use_auth_token=True, split="val")
val_ds

In [ ]:
records = list()
for example in tqdm(val_ds):
    record = dict(
        answer=example["answers"]["text"][0],
        context=example["context"],
        texas_qa=texas_qa(context=example["context"], question=example["question"])["answer"].strip('.,()\'"*†'),
        botxo_qa=botxo_qa(context=example["context"], question=example["question"])["answer"].strip('.,()\'"*†'),
        scandi_qa=scandi_qa(context=example["context"], question=example["question"])["answer"].strip('.,()\'"*†'),
    )
    records.append(record)

In [ ]:
df = pd.DataFrame.from_records(records)
df.head()

In [ ]:
wrong_texas_df = df.query('answer != texas_qa and answer == scandi_qa')
wrong_texas_df.head()

In [ ]:
load_dataset("dane")